In [1]:
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time
import os
tf.disable_v2_behavior()
print(np.array([['X11','X12','X13','X14'],['X21','X22','X23','X24']]).shape)
posdata = np.stack(([['X11','X12','X13','X14'],['X21','X22','X23','X24']],[['NN11','NN12','NN13','NN14'],['NN21','NN22','NN23','NN24']],[['NNN11','NNN12','NNN13','NNN14'],['NNN21','NNN22','NNN23','NNN24']]),axis=2)
   
posdata=np.reshape(posdata,(2,4*3))
print(posdata)
posdata=np.reshape(posdata[0],(4,3))
print(posdata)

Instructions for updating:
non-resource variables are not supported in the long term
(2, 4)
[['X11' 'NN11' 'NNN11' 'X12' 'NN12' 'NNN12' 'X13' 'NN13' 'NNN13' 'X14'
  'NN14' 'NNN14']
 ['X21' 'NN21' 'NNN21' 'X22' 'NN22' 'NNN22' 'X23' 'NN23' 'NNN23' 'X24'
  'NN24' 'NNN24']]
[['X11' 'NN11' 'NNN11']
 ['X12' 'NN12' 'NNN12']
 ['X13' 'NN13' 'NNN13']
 ['X14' 'NN14' 'NNN14']]


In [2]:
def load_data(file_name,maxrow):
    print("read data from \"",file_name,"\"")
    
    xdata=np.loadtxt(file_name,max_rows=maxrow)
    L=(xdata.shape[1]-3)//9
    print(L)
    ydata=xdata[:,1]
    dxdata=xdata[:,xdata.shape[1]-1]
    
    posdata=xdata[:,2+4*L:2+5*L]
    NNdata=np.stack((xdata[:,2+L:2+2*L],xdata[:,2+3*L:2+4*L],xdata[:,2+5*L:2+6*L],xdata[:,2+7*L:2+8*L]),axis=2)
    NNNdata=np.stack((xdata[:,2:2+L],xdata[:,2+2*L:2+3*L],xdata[:,2+6*L:2+7*L],xdata[:,2+8*L:2+9*L]),axis=2)
    NNdata=np.mean(NNdata,axis=2)
    NNNdata=np.mean(NNNdata,axis=2)
    
    averagex=np.mean(xdata[:,2:xdata.shape[1]-1])
    print(averagex)
    gdata_append=0.25*(posdata+np.outer(dxdata,np.ones(L))-averagex)**4-averagex**2/2.0*(posdata+np.outer(dxdata,np.ones(L))-averagex)**2-0.25*(posdata-averagex)**4+averagex**2/2.0*(posdata-averagex)**2
    print(np.outer(dxdata,np.ones(L)).shape)
    gdata_append=np.mean(gdata_append,axis=1)
    zdata_append=np.sum((posdata+np.outer(dxdata,np.ones(L)))**2-posdata**2,axis=1)
    print(posdata.shape)
    posdata = np.stack((posdata,NNdata,NNNdata),axis=2)
   
    posdata=np.reshape(posdata,(maxrow,L*3))
    
    return posdata, dxdata, ydata, zdata_append, gdata_append


def scaled_data(xdata):
    xdata_mean=np.mean(xdata,axis=0)
    xdata_std=np.std(xdata,axis=0)
    xdata_scaled= (xdata-xdata_mean)/xdata_std
    return xdata_scaled, xdata_mean, xdata_std


def get_next_batch(x,y,start,end):
    x_batch=x[start:end,0:x.shape[1]-3]
    dx_batch=x[start:end,x.shape[1]-3]
    z_batch=x[start:end,x.shape[1]-2]
    g_batch=x[start:end,x.shape[1]-1]
    y_batch=y[start:end]
    return x_batch, dx_batch, z_batch, g_batch, y_batch


def weight_variable(name, shape):
    initer=tf.truncated_normal_initializer(stddev=0.02)
    return tf.get_variable('W_'+name, dtype=tf.float32,shape=shape,initializer=initer)

def bias_variable(name, shape):
    initial=tf.constant(0.0,shape=shape,dtype=tf.float32)
    return tf.get_variable('b_'+name,dtype=tf.float32,initializer=initial)

def create_new_conv_layer(input_data, deltax, num_filters, filter_shape, name):
    # setup the filter input shape for tf.nn.conv_2d
    conv_filt_shape = [filter_shape[0], filter_shape[1], 1, num_filters]

    # initialise weights and bias for the filter
    weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev=0.03),
                                      name=name+'_W')
    bias = tf.Variable(tf.truncated_normal([num_filters]), name=name+'_b')

    # setup the convolutional layer operation
    out_layer = tf.nn.conv2d(input_data, weights, [1, 3, 1, 1],padding='VALID')

    # add the bias
    out_layer += bias
    
    # add the theta term
    in_dim=dx.get_shape()[1]
    theta = tf.Variable(tf.truncated_normal([in_dim,num_filters]), name=name+'_theta')
    print_out_layer=tf.Print(out_layer,[tf.shape(out_layer)],"This is the shape of that out_layer, should be 100, 12,1,5: ",summarize=4)
    print_out_layer +=tf.tensordot(tf.tensordot(tf.squeeze(dx),tf.ones([12],tf.float32),axes=0),theta,axes=0)

    # apply a sigmoid non-linear activation
    print_out_layer = tf.nn.sigmoid(print_out_layer)

    
    return print_out_layer, weights, bias, theta





def fc_layer(x,num_units,name,activation):
    in_dim=x.get_shape()[1]
    W=weight_variable(name,shape=[in_dim,num_units])
    b=bias_variable(name,[num_units])
    layer=tf.matmul(x,W)
    layer+=b
    if activation=='softplus':
        layer=tf.nn.softplus(layer)
    elif activation=='sigmoid':
        layer=tf.nn.sigmoid(layer)
    return layer, W, b

def fc_layer_final(x,g,z,num_units,name):
    in_dim=x.get_shape()[1]
    W1=weight_variable(name,shape=[in_dim,num_units])
    W2g=weight_variable(name+'g',shape=[1,num_units])
    W3z=weight_variable(name+'x',shape=[1,num_units])
    b=bias_variable(name,[num_units])
    print_x=tf.Print(x,[tf.shape(x)],"This is the shape of the x value: ")
    print_g=tf.Print(g,[g],"This is the g value: ")
    print_z=tf.Print(z,[z],"This is the z value: ")
    print_W3z=tf.Print(W3z,[W3z],"This is the W3z value: ")
    
    layer=tf.matmul(print_x,W1)+tf.matmul(print_g,W2g)+tf.matmul(print_z,print_W3z)

    print_layer=tf.Print(layer,[layer],"This is the pre b layer value: ")
    print_layer+=b
    return print_layer, W1, W2g, print_W3z, b

def save_data(file_name,name,x,matrix):
    out_file=open(file_name,'a')
    if matrix:
        out_file.write("%30s: [  %4d  ,  %4d  ]\n"%(name,x.shape[0], x.shape[1]))
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                out_file.write("%10.6f    " %(x[i,j]))
            out_file.write("\n")
    else:
        out_file.write("%s: [  %4d  ]\n"%(name,x.shape[0]))
        for i in range(x.shape[0]):
            out_file.write("%10.6f    "%(x[i]))
        out_file.write("\n")
    out_file.close()

In [3]:
# set parameters
file_name="./traindata/build/globaltrain.dat"
#file_name="./Desktop/block_train0003.dat"

maxrow=80001

# set hyper parameters
epochs=50
batch_size=100
display_freq=1
learning_rate=0.001
L2_regularization=0.001

In [4]:
# read in data
xdata, dxdata, ydata, zdata_append, gdata_append=load_data(file_name,maxrow)

# scaled data
xdata_scaled,        xdata_mean,        xdata_std        = scaled_data(xdata)
zdata_append_scaled, zdata_append_mean, zdata_append_std = scaled_data(zdata_append)
gdata_append_scaled, gdata_append_mean, gdata_append_std = scaled_data(gdata_append)
dxdata_scaled,       dxdata_mean,       dxdata_std       = scaled_data(dxdata)
xdata_combined=np.concatenate((xdata_scaled,np.array([dxdata_scaled]).T,np.array([zdata_append_scaled]).T,np.array([gdata_append_scaled]).T),axis=1)

# split data
x_train, x_test, y_train, y_test = train_test_split(xdata_combined, ydata, test_size=0.33, random_state=42)
L = (x_train.shape[1]-3)//3
print("Size of:")
print('x_train:\t{}'.format(x_train.shape))
print('y_train:\t{}'.format(y_train.shape))
print('x_test:\t{}'.format(x_test.shape))
print('y_test:\t{}'.format(y_test.shape))

read data from " ./traindata/build/globaltrain.dat "
41
-3.999213336841781
(80000, 41)
(80000, 41)
Size of:
x_train:	(53600, 126)
y_train:	(53600,)
x_test:	(26400, 126)
y_test:	(26400,)


In [14]:
ydata = np.delete(ydata,np.argwhere(np.isnan(ydata)))
print(np.argwhere(np.isnan(ydata)))

[]


In [6]:
#create placeholders
x = tf.placeholder(tf.float32,shape=[None, x_train.shape[1]-3],name='X')
x_shaped = tf.reshape(x,[-1,L,3,1])
dx = tf.placeholder(tf.float32,shape=[None, 1] ,name='dX')
g = tf.placeholder(tf.float32,shape=[None, 1] ,name='g')
z = tf.placeholder(tf.float32,shape=[None, 1] ,name='z')
y = tf.placeholder(tf.float32,shape=[None,1] ,name='Y')

#define the network
convout, Wconv, bconv, thetaconv = create_new_conv_layer(x_shaped, dx, 1, [8,3], name="convlayer")
flattened = tf.reshape(convout, [-1, 1*((L-8)//3+1)])
fc1, W1, b1 = fc_layer(flattened, 10 , 'FC1' , 'sigmoid')
output, W2, W2g, W2z, b2 = fc_layer_final(fc1,g,z,1,'FC2')

print_output = tf.Print(output,[output],"output is: ")
print_y = tf.Print(y,[y],"actual value is: ")

loss=tf.reduce_mean(tf.squared_difference(print_output, print_y))
regularizer=tf.reduce_sum([tf.nn.l2_loss(W1), tf.nn.l2_loss(W2), tf.nn.l2_loss(W2g), tf.nn.l2_loss(W2z), tf.nn.l2_loss(Wconv), tf.nn.l2_loss(thetaconv)])
total_loss=tf.reduce_sum(loss+L2_regularization*regularizer)
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate,name="Adam_op").minimize(total_loss)

Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:

Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().


In [7]:
# initializing all variables
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
num_tr_iter=int(len(y_train)/batch_size)
print('num_tr_iter = ', num_tr_iter)
store_loss_train_name='global_loss_train.dat'
sltn=open(store_loss_train_name,"w+")

for epoch in range(epochs):
    print('-----------------------------------------------')
    print('Training epoch : ', epoch)
    print('-----------------------------------------------')
    for iteration in range(num_tr_iter):
        start=iteration*batch_size
        end=(iteration+1)*batch_size
        x_batch,dx_batch,z_batch,g_batch,y_batch=get_next_batch(x_train,y_train,start,end) 
        z_batch=np.expand_dims(z_batch,1)
        dx_batch=np.expand_dims(dx_batch,1)
        g_batch=np.expand_dims(g_batch,1)
        y_batch=np.expand_dims(y_batch,1)

        feed_dict_batch={x:x_batch,dx:dx_batch,g:g_batch,z:z_batch,y:y_batch}

        sess.run(optimizer,feed_dict=feed_dict_batch)

        if iteration%display_freq==0:
            loss_batch=sess.run(loss,feed_dict=feed_dict_batch)
            print("iter {0:3d}:\t Loss = {1:2f}".format(iteration,loss_batch))
            sltn.write("%6d    %15.12f\n" %(iteration+epoch*num_tr_iter, loss_batch))
sltn.close()

# plot training loss function
#matplotlib.figure.Figure()
xloss,yloss=np.loadtxt(store_loss_train_name,unpack=True)
plt.plot(xloss, yloss)
plt.xlabel('Step')
plt.ylabel('Loss')
plt.show()

# save and plot y_predict_train, y_train
store_y_train_name='y_train.dat'
sytn=open(store_y_train_name,"w+")
np.random.seed(68721)
random_list=np.random.randint(len(y_train),size=2000)
x_batch=np.zeros([1000,x_train.shape[1]-1])
xprime_batch=np.zeros([1000,1])
y_batch=np.zeros([1000,1])
for i in range(1000):
    x_batch[i,0:x_train.shape[1]-1]=x_train[random_list[i],0:x_train.shape[1]-1].T
    xprime_batch[i,0]=x_train[random_list[i],x_train.shape[1]-1]
    y_batch[i,0]=y_train[random_list[i]]

feed_dict_batch={x:x_batch,xprime:xprime_batch,y:y_batch}
y_train_predict=sess.run(output,feed_dict=feed_dict_batch)
print(y_train_predict.shape)
for i in range(1000):
    sytn.write("%15.12f    %15.12f\n" %(y_batch[i,0], y_train_predict[i,0]))
sytn.close()

#matplotlib.figure.Figure()
y1, y2=np.loadtxt(store_y_train_name,unpack=True)
plt.plot(y1, y2,'ro')
plt.plot(y1,y1,'-b')
plt.xlabel('Y Value')
plt.ylabel('Y Prediction')
plt.show()

num_tr_iter =  53
-----------------------------------------------
Training epoch :  0
-----------------------------------------------
iter   0:	 Loss = nan
iter   1:	 Loss = nan
iter   2:	 Loss = nan
iter   3:	 Loss = nan
iter   4:	 Loss = nan
iter   5:	 Loss = nan
iter   6:	 Loss = nan
iter   7:	 Loss = nan
iter   8:	 Loss = nan
iter   9:	 Loss = nan
iter  10:	 Loss = nan
iter  11:	 Loss = nan
iter  12:	 Loss = nan
iter  13:	 Loss = nan
iter  14:	 Loss = nan
iter  15:	 Loss = nan
iter  16:	 Loss = nan
iter  17:	 Loss = nan
iter  18:	 Loss = nan
iter  19:	 Loss = nan
iter  20:	 Loss = nan
iter  21:	 Loss = nan
iter  22:	 Loss = nan
iter  23:	 Loss = nan
iter  24:	 Loss = nan
iter  25:	 Loss = nan
iter  26:	 Loss = nan
iter  27:	 Loss = nan
iter  28:	 Loss = nan
iter  29:	 Loss = nan
iter  30:	 Loss = nan
iter  31:	 Loss = nan
iter  32:	 Loss = nan
iter  33:	 Loss = nan
iter  34:	 Loss = nan
iter  35:	 Loss = nan
iter  36:	 Loss = nan
iter  37:	 Loss = nan
iter  38:	 Loss = nan
iter  39

iter  18:	 Loss = nan
iter  19:	 Loss = nan
iter  20:	 Loss = nan
iter  21:	 Loss = nan
iter  22:	 Loss = nan
iter  23:	 Loss = nan
iter  24:	 Loss = nan
iter  25:	 Loss = nan
iter  26:	 Loss = nan
iter  27:	 Loss = nan
iter  28:	 Loss = nan
iter  29:	 Loss = nan
iter  30:	 Loss = nan
iter  31:	 Loss = nan
iter  32:	 Loss = nan
iter  33:	 Loss = nan
iter  34:	 Loss = nan
iter  35:	 Loss = nan
iter  36:	 Loss = nan
iter  37:	 Loss = nan
iter  38:	 Loss = nan
iter  39:	 Loss = nan
iter  40:	 Loss = nan
iter  41:	 Loss = nan
iter  42:	 Loss = nan
iter  43:	 Loss = nan
iter  44:	 Loss = nan
iter  45:	 Loss = nan
iter  46:	 Loss = nan
iter  47:	 Loss = nan
iter  48:	 Loss = nan
iter  49:	 Loss = nan
iter  50:	 Loss = nan
iter  51:	 Loss = nan
iter  52:	 Loss = nan
-----------------------------------------------
Training epoch :  7
-----------------------------------------------
iter   0:	 Loss = nan
iter   1:	 Loss = nan
iter   2:	 Loss = nan
iter   3:	 Loss = nan
iter   4:	 Loss = nan
iter

KeyboardInterrupt: 